In [1]:
import re
import pandas as pd
import numpy as np
import json
import subprocess
import glob
import os

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
matplotlib.style.use('ggplot')
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch

In [2]:
def set_config_path():
    global test_mode
    global data_mode
    global mode
    global model_file
    global save_path
    global w_file
    global sem_file
    global ho_file
    
    test_mode = 'BaseVer02'
    data_mode = 'CA'
    mode = 'NEW'
    model_file = '..\\..\\ABMIRISLab\\Examples\\BaseModel\\' + test_mode + '\\' + data_mode + '\\basemodel.model'
    save_path = '..\\..\\ABMIRISLab\\Saves\\Jan-11-2018\\BaseModel\\' + test_mode + '\\' + data_mode + '\\basemodel.model'

    w_file = '..\\..\\ABMIRISLab\\Examples\\BaseModel\\' + test_mode + '\\' + data_mode + '\\w.json'
    sem_file = '..\\..\\ABMIRISLab\\Examples\\BaseModel\\' + test_mode + '\\' + data_mode + '\\sem.json'
    ho_file = '..\\..\\ABMIRISLab\\Examples\\BaseModel\\' + test_mode + '\\' + data_mode + '\\ho.json'

In [8]:
el_price = 0.15

In [9]:
def update_config_for_params(adjustment):
    #update path to save file 
    #open file at path 
    #rewrite it 
    #'path_so_save=' + save_path
    #'mode=NEW'

    with open(model_file, 'w') as model_file_:
        model_file_.write('path_to_save={}\n'.format(save_path))
        model_file_.write('mode={}\n'.format('NEW'))

    with open(w_file) as infile:
        w_setup = json.load(infile)
        for setting in w_setup['WorldSettings']['params_exog']:
            if 'EParamTypes::ScenarioEfficiencyAdjustment' in setting:
                setting['EParamTypes::ScenarioEfficiencyAdjustment'] = adjustment
            if 'EParamTypes::ElectricityPriceUCDemand' in setting:
                setting['EParamTypes::ElectricityPriceUCDemand'] = el_price
            if 'EParamTypes::ElectricityPriceUCSupply' in setting:
                setting['EParamTypes::ElectricityPriceUCSupply'] = el_price

        with open(w_file, 'w') as outfile:
            json.dump(w_setup, outfile)

    with open(sem_file) as infile:
        sem_setup = json.load(infile)
        if (adjustment > 1.0):
    #        sem_setup['costs_base'] = 0.39 * (1 + (adjustment - 1) / 2)
            sem_setup['costs_base'] = 0.6 
        else:
            sem_setup['costs_base'] = 0.6

        for setting in sem_setup['params']:
            if 'EParamTypes::SEMEfficiencyUpgradeResearch' in setting:
                setting['EParamTypes::SEMEfficiencyUpgradeResearch'] = 0.0

        with open(sem_file, 'w') as outfile:
            json.dump(sem_setup, outfile)


    with open(ho_file) as infile:
        ho_setup = json.load(infile)
        ho_setup['House']['roof_effective_size'] = 'FORMULA::0.25'

        with open(ho_file, 'w') as outfile:
            json.dump(ho_setup, outfile)
    

In [10]:
def simulation_run_for_params(adjustment):
    for test_mode in ['BaseVer02']:
        for data_mode in ['CA']:

            file = '..\\..\\ABMIRISLab\\Examples\\BaseModel\\' + test_mode + '\\' + data_mode + '\\w.json'


            mode="NEW"
            model_file='..\\..\\ABMIRISLab\\Examples\\BaseModel\\' + test_mode + '\\' + data_mode + '\\basemodel.model'


            #update path to save file 
            save_path = '..\\..\\ABMIRISLab\\Saves\\Jan-11-2018\\BaseModel\\' + test_mode + \
                        '\\' + data_mode + '\\' + str(adjustment) + '\\basemodel.model'


            #check that save path exists 
            directory = os.path.dirname(save_path)
            if not os.path.exists(directory):
                os.makedirs(directory)
                
                
            #open file at path 
            #rewrite it 
            #'path_so_save=' + save_path
            #'mode=NEW'

            with open(model_file, 'w') as model_file_:
                model_file_.write('path_to_save={}\n'.format(save_path))
                model_file_.write('mode={}\n'.format('NEW'))


            with open(file) as infile:
                w_setup = json.load(infile)

            for seed in range(100, 110):
                w_setup['SEED'] = seed
                w_setup['ParametersCode'] = test_mode + '_' + data_mode + '_' + str(adjustment)

                with open(w_file, 'w') as outfile:
                    json.dump(w_setup, outfile)
                
                #run simulation 
                path = 'BaseModel.exe'
                args = [path, mode, model_file]
                res = -1

                i = 0 
                while ((res != 0) and (i < 2)):
                    try:
                        subprocess.call([path, mode, model_file])
                        i = 0
                        res = 0
                    except subprocess.CalledProcessError as e:
                        i = i + 1
                        res = -1

In [11]:
set_config_path()

In [14]:
for adjustment in np.linspace(1.0,1.2,5,endpoint=True):
    update_config_for_params(adjustment)
    
    #run simulation for params
    simulation_run_for_params(adjustment)
    
    #get aggregate data to show installs

In [13]:
for adjustment in np.linspace(1.0,1.1,1,endpoint=False):
    update_config_for_params(adjustment)
    
    #run simulation for params
    simulation_run_for_params(adjustment)
    
    #get aggregate data to show installs

# Testing options

run for seeds, save into folder? with efficiency step? 
locate log files - where they save? 


seeds - efficiency step - 

costs try 

In [13]:
adjustment = 1.0
for test_mode in ['BaseVer01']:
    for data_mode in ['CA']:
        save_path = '..\\..\\ABMIRISLab\\Saves\\Aug-29-2017\\BaseModel\\' + test_mode + \
                        '\\' + data_mode + '\\' + str(adjustment) + '\\basemodel.model'

        print(os.path.dirname(save_path))
        directory = os.path.dirname(save_path)
        if not os.path.exists(directory):
            os.makedirs(directory)


..\..\ABMIRISLab\Saves\Aug-29-2017\BaseModel\BaseVer01\CA\1.0


In [ ]:
update_config_for_params(1.0)

In [ ]:
path = 'BaseModel.exe'
args = [path, mode, model_file]
subprocess.call([path, mode, model_file])